In [251]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/train_weather.csv
/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/test_weather.csv
/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/submission.csv
/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/train.csv
/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/test.csv


In [252]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [253]:
train_df = pd.read_csv("/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/train.csv")
test_df = pd.read_csv("/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/test.csv")
train_df_weather = pd.read_csv("/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/train_weather.csv")
test_df_weather = pd.read_csv("/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/test_weather.csv")
submission_df = pd.read_csv("/kaggle/input/dare-in-reality-hackathon-2021-machine-hack/submission.csv")

In [254]:
print('Size of Train Data : {} \r\nSize of Train Weather Data : {} '.format(train_df.shape,train_df_weather.shape))
print("--------------------------------------------------------------------------")
print('Size of Test Data : {} \r\nSize of Test Weather Data : {} '.format(test_df.shape,test_df_weather.shape))

Size of Train Data : (10276, 25) 
Size of Train Weather Data : (914, 11) 
--------------------------------------------------------------------------
Size of Test Data : (420, 25) 
Size of Test Weather Data : (167, 11) 


In [255]:
train_df.columns = train_df.columns.str.replace(' ','')
train_df_weather.columns = train_df_weather.columns.str.replace(' ','')
test_df.columns = test_df.columns.str.replace(' ','')
test_df_weather.columns = test_df_weather.columns.str.replace(' ','')

In [256]:
df_train = train_df
df_test = test_df

In [257]:
df_train.isnull().sum()

NUMBER                            0
DRIVER_NUMBER                     0
LAP_NUMBER                        0
LAP_TIME                          0
LAP_IMPROVEMENT                   0
CROSSING_FINISH_LINE_IN_PIT    9042
S1                                4
S1_IMPROVEMENT                    0
S2                                8
S2_IMPROVEMENT                    0
S3                               34
S3_IMPROVEMENT                    0
KPH                              30
ELAPSED                           0
HOUR                              0
S1_LARGE                          4
S2_LARGE                          8
S3_LARGE                         34
DRIVER_NAME                       0
PIT_TIME                       8378
GROUP                          9768
TEAM                              0
POWER                          7373
LOCATION                          0
EVENT                             0
dtype: int64

In [258]:
def time2seconds(time):
  if type(time) != str:
    return time
  parts = [float(p) for p in time.split(':')]
  parts = [p * (60 ** i) for i, p in enumerate(reversed(parts))]
  return sum(parts)

In [259]:
time_cols = ['ELAPSED', 'S1', 'S2', 'S3', 'S1_LARGE', 'S2_LARGE', 'S3_LARGE', 'HOUR','PIT_TIME']
for col in time_cols:
  df_train[col] = df_train[col].apply(time2seconds)
  df_test[col] = df_test[col].apply(time2seconds)

df_train.drop(['CROSSING_FINISH_LINE_IN_PIT','GROUP'],axis=1,inplace=True)
df_test.drop(['CROSSING_FINISH_LINE_IN_PIT','GROUP','LAP_TIME'],axis=1,inplace=True)

**Run the Below Code two times.**

In [260]:
missing_val_cols = list()
def miss_val_col(all_cols):
  missing_val = list()
  for col in all_cols:
    if df_train[col].isnull().sum() > 0:
      missing_val.append(col)
    else:
      continue
  if len(missing_val) == 0:
    return f"No missing values found"
  else:
    for i in missing_val:
      df_train[i] = df_train[i].fillna(method='bfill')
      df_train[i] = df_train[i].fillna(method='ffill')
    for i in missing_val:
      df_test[i] = df_test[i].fillna(method='ffill')
      df_test[i] = df_test[i].fillna(method='bfill')
    return missing_val
print(miss_val_col(df_train.columns))
print(miss_val_col(df_test.columns))

['S1', 'S2', 'S3', 'KPH', 'S1_LARGE', 'S2_LARGE', 'S3_LARGE', 'PIT_TIME', 'POWER']
No missing values found


In [261]:
x = []
for i in df_test.select_dtypes(include='O').columns.values:
  x.append(i)
lbl_encoder = LabelEncoder()
for col in x:
  df_train[col] = lbl_encoder.fit_transform(df_train[col])
  df_test[col] = lbl_encoder.transform(df_test[col])

In [262]:
target = df_train['LAP_TIME']
df_train.drop(['LAP_TIME'],axis=1,inplace=True)

In [263]:
X_train, X_test, y_train, y_test = train_test_split(df_train,target,test_size=0.25)

In [264]:
minmax = MinMaxScaler()
for te_col in df_test.columns:
  df_train[te_col] = minmax.fit_transform(df_train[[te_col]])
  df_test[te_col] = minmax.transform(df_test[[te_col]])

In [265]:
model = LinearRegression()
model. fit(X_train,y_train)
pred = model.predict(X_test)

In [266]:
np.sqrt(mean_squared_log_error(y_test, pred)) # with min max scalar

0.5153260967113896

In [267]:
prediction = model.predict(df_test)
prediction = pd.DataFrame(prediction, columns=['LAP_TIME'])
prediction.to_csv('submission_file_scale.csv', index=False)